**Notebook content:**
- 
- 

In [1]:
%reset -f 

In [2]:
import sys
import pandas as pd
import numpy as np
import datetime
from IPython.display import display

pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 200)
#pd.set_option('display.float_format', lambda x: '%.f' % x)
pd.options.display.precision = 4

#import sys
np.set_printoptions(threshold=sys.maxsize) #- print the full NumPy array

from myDefs.defs import *

# visualization
import seaborn as sns
import matplotlib.pyplot as plt 
from matplotlib.pyplot import figure
%matplotlib inline


In [3]:
nValue = 'nValue' #numeric value

**Read events file**

In [4]:
file = DATA_PATH + "parseData1.csv"
df = pd.read_csv(file, sep=',')  

In [5]:
df.shape

(1683616, 17)

In [6]:
df.head();

**Read Anonymous file**

In [7]:
output_path = "{}parseAnonymous0.csv".format(DATA_PATH)
anonymous = pd.read_csv(output_path, sep=',')

In [8]:
anonymous.head();

In [9]:
anonymousIDs = anonymous["pid"]

In [10]:
dfIDs = df["pid"]

In [11]:
anonymous.nunique()

pid                        680
Days of hospitalization     47
Sex                          2
Age                         73
Date of surgery            205
Patient classification      10
Name of surgery            313
Responsible surgeon         17
Urgency                      2
Surgical approach            3
Complications               90
Kk                           1
Simple                       1
Severe                       2
dtype: int64

In [12]:
s1 = set(dfIDs) 
s2 = set(anonymousIDs)
intersection = list(s1.intersection(s2))

In [13]:
len(intersection)

680

**Read Events_map file**

In [14]:
path = "{}organizeEvents/all_events_manual.xlsx".format(DATA_PATH)
events = pd.read_excel(path)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/hp4mp/PythonProjects/AnastomoticLeakProject/Data/organizeEvents/all_events_manual.xlsx'

In [ ]:
events.head()

In [ ]:
names = events.set_index('eventName')['feature_name']
names = names[names.notna()]

dnames = names.to_dict()
dnames

In [ ]:
names.index

In [ ]:
sub_events = df.loc[df['eventName'].isin(names.index), :]

sub_events.loc[:,'eventName'] = sub_events['eventName'].replace(dnames)
sub_events

# Feature by Feature

In [ ]:
features_df = []

In [ ]:
def create_feature(df, feature_name, alter_names, column_value='dValue', print_output=True):

    '''
    df - data frame of all events 
    feature_name - name of feature 
    alter_names - alternative names of feature
    
    '''
   
    #get the required events
    sub_events = df[df["eventName"].isin(alter_names)]

    #select only rellevant columns 
    feature = sub_events.loc[:, ['pid', 'eventStartDate', 'eventEndDate', 'unitOfMeasure']]
    
    #get values of event 
    feature[feature_name] = sub_events[column_value]
    
    if print_output:
        
        print("Num {} events:".format(feature_name), sub_events.shape[0])     

        print("\nHas sValue:", sub_events['sValue'].any())
        print("Has event description:", sub_events['eventDesc'].any())

        print("\nDescribe numeric values:\n", sub_events[['dValue', 'iValue']].describe())

        print("\nUnit of measure:", sub_events['unitOfMeasure'].unique())
    
        print("\nResult:")
        display(feature.head())


    return feature


## Body heat

In [ ]:
alter_names = ['חום.', 'חום', 'Temperature', 'DATEX.body temp.value_1', 'DATEX.body temp.value_2']
body_heat = create_feature(df, 'body heat', alter_names)

In [ ]:
def fahrenheit_to_celsius(f):
    return (f - 32) * 5.0/9.0

#.apply(fahrenheit_to_celsius)
fahrenheits = body_heat['unitOfMeasure']=='F'
body_heat.loc[fahrenheits, 'body heat'] = body_heat.loc[fahrenheits, 'body heat'].apply(fahrenheit_to_celsius)
body_heat.loc[fahrenheits, 'unitOfMeasure'] = 'C'

body_heat[body_heat['body heat'] > 40];

# Create A Features Table

In [ ]:
'''
Adding suffix to duplicate patients (pateints who underwent > 1 surgery) 
'''

res = anonymous.copy()
anonymous.head()
res = res[['pid', 'Sex', 'Age', 'Date of surgery']]

#convert column to strig 
res['pid'] = res['pid'].astype(str)

#Number each item in each group from 1 to the length of that group.
cumcount = res.groupby('pid').cumcount() + 1


#which patients are duplicated
idx = res['pid'].duplicated(keep=False)

#concatenate suffixes
res.loc[idx, 'pid'] = res['pid'] + "_" + cumcount.astype(str)

res[idx]


In [ ]:
ann.groupby('pid').apply(lambda x:np.arange(len(x)))

In [ ]:
ann['cumcount'] = ann.groupby('pid').cumcount()
ann['pid'].astype(str) +"_"+ ann['cumcount'].astype(str)

In [ ]:
def f(df):
    return df.to_dict('list')

def createEventsDic(df):
    #group by event name 
    cols = ["value", "unitOfMeasure", "eventDesc"]
    x = df.groupby("eventName")[cols].apply(f).to_dict()
    return x
    
def getEventsByTimeRange(df, start_time, end_time):
    '''
    Get events in the range [startTime, endTime)
    '''
    return df[df['eventStartDate'] >= start_time & df['eventStartDate'] < end_time]
    
    
    
def nomi(df):
    
    #get surgery date of current patient 
    pid = df.iloc[0]["pid"]
    surgery_date = dates.loc[pid, "surgery_1"]
    
    pass
    
   
    titels = ["01_day_before_surgery"]
    
    if pd.isnull(surgery_date):
        return None
        #t = [0] * len(titels)
     
    #dates before and surgery
   
    

    #create data frame
    data = []
    df = pd.DataFrame([data], columns=titels)  
    
    return df

    

In [ ]:
createEventsDic(df.head())

In [ ]:
df.groupby('pid').apply()

In [ ]:
df[df["eventName"] == "NoEventName"].shape[0]

In [ ]:
patients_ids = df.groupby(eventDesc).groups.keys()

In [ ]:
df.head()

In [ ]:
pd.options.display.max_colwidth = 100


In [ ]:
df[df['eventName']=='Fever']

In [20]:
df[df['eventName']=='כן']

,pid,admissionId,eventName,eventStartDate,eventEndDate,dValue,iValue,sValue,eventDesc,unitOfMeasure,orderNumber,organismId,eventCode,eventCodeOrg,eventType,eventTypeOrg,sourceName
165,4.8546e+08,1840978.0,כן,2018-02-18 09:13:08.9300000,2018-02-18 09:13:08.9300000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,175853000,כן,snomed,conv_device_type_cham,[MIRROR_Chameleon].[Venous_Catheter_Dynamic_Qu...
953,1.7782e+08,1935793.0,כן,2018-12-09 15:41:11.5770000,2018-12-09 15:41:11.5770000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,175853000,כן,snomed,conv_device_type_cham,[MIRROR_Chameleon].[Venous_Catheter_Dynamic_Qu...
3881,1.0133e+09,1805375.0,כן,2017-07-29 14:28:03.2730000,2017-07-29 14:28:03.2730000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,175853000,כן,snomed,conv_device_type_cham,[MIRROR_Chameleon].[Venous_Catheter_Dynamic_Qu...
4079,1.0133e+09,1805375.0,כן,2017-07-31 04:11:13.6730000,2017-07-31 04:11:13.6730000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,175853000,כן,snomed,conv_device_type_cham,[MIRROR_Chameleon].[Venous_Catheter_Dynamic_Qu...
4320,1.0133e+09,1840641.0,כן,2018-02-18 22:44:23.0530000,2018-02-18 22:44:23.0530000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,175853000,כן,snomed,conv_device_type_cham,[MIRROR_Chameleon].[Venous_Catheter_Dynamic_Qu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1675468,1.0726e+08,1927892.0,כן,2018-10-28 13:01:00.6400000,2018-10-28 13:01:00.6400000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,175853000,כן,snomed,conv_device_type_cham,[MIRROR_Chameleon].[Venous_Catheter_Dynamic_Qu...
1680427,3.4132e+08,1922171.0,כן,2018-09-27 15:11:47.6270000,2018-09-27 15:11:47.6270000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,175853000,כן,snomed,conv_device_type_cham,[MIRROR_Chameleon].[Venous_Catheter_Dynamic_Qu...
1682058,3.4132e+08,1922794.0,כן,2018-09-30 13:30:46.9100000,2018-09-30 13:30:46.9100000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,175853000,כן,snomed,conv_device_type_cham,[MIRROR_Chameleon].[Venous_Catheter_Dynamic_Qu...
1682396,1.0726e+08,1925979.0,כן,2018-10-18 14:37:57.5500000,2018-10-18 14:37:57.5500000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,175853000,כן,snomed,conv_device_type_cham,[MIRROR_Chameleon].[Venous_Catheter_Dynamic_Qu...
